In this notebook, I will preprocess the Mannens et al. dataset for the application\
of Epiregulon package to infer eGRNs using all of the genes.

In [1]:
getwd()

[1] "/fast/AG_Bunina/Yusuf/Project_Endothelial_and_Stroke/Datasets/Chromatin_and_Gene_Exp/2024_C_A_Mannens_C_et_al/04_02_25"

In [2]:
.libPaths()

[1] "/fast/home/y/ykazci/R/x86_64-unknown-linux-gnu-library/4.3"               
[2] "/gnu/store/mj2kqa65bbqr3155zccvm01hq7wi7fng-profile/site-library"         
[3] "/gnu/store/29x2k7i71g9xq09xmbj1lk515cl7if63-r-minimal-4.4.2/lib/R/library"

In [4]:
# load the R environment with the necessary packages:

renv::load()

In [5]:
renv::status()

No issues found -- the project is in a consistent state.


In [6]:
library(Seurat)

Loading required package: SeuratObject

Loading required package: sp

‘SeuratObject’ was built under R 4.4.1 but the current version is
4.4.2; it is recomended that you reinstall ‘SeuratObject’ as the ABI
for R may have changed


Attaching package: ‘SeuratObject’


The following objects are masked from ‘package:base’:

    intersect, t




In [7]:
library(Signac)

In [8]:
mannens_et_al_seurat <- readRDS(here::here('r_objects', 'mannens_et_al_seurat_obj.RDS'))

In [9]:
mannens_et_al_seurat

An object of class Seurat 
430192 features across 49470 samples within 2 assays 
Active assay: RNA (25071 features, 5000 variable features)
 3 layers present: counts, data, scale.data
 1 other assay present: peaks

In [10]:
GeneExpressionMatrix <- as.SingleCellExperiment(mannens_et_al_seurat, assay="RNA")

In [11]:
GeneExpressionMatrix

class: SingleCellExperiment 
dim: 25071 49470 
metadata(0):
assays(3): counts logcounts scaledata
rownames(25071): MALAT1 AUTS2 ... APOL5 SLURP1
rowData names(0):
colnames(49470): 10X280_1:CAGATTCAGCAGCTCA 10X365_2:ACCAATATCAATGACC
  ... 10X346_4:CGCTGTGCACGTAATT 10X406_4:GAGAAACGTGGTGAGA
colData names(89): orig.ident nCount_RNA ... main_cell_types ident
reducedDimNames(0):
mainExpName: RNA
altExpNames(0):

In [12]:
library(SingleCellExperiment)

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, rowMedians, rowMins,
    rowOrderStats, rowProds, rowQuantiles, rowRanges

In [13]:
GeneExpressionMatrix |> rowRanges()

GRangesList object of length 25071:
$MALAT1
GRanges object with 0 ranges and 0 metadata columns:
   seqnames    ranges strand
      <Rle> <IRanges>  <Rle>
  -------
  seqinfo: no sequences

$AUTS2
GRanges object with 0 ranges and 0 metadata columns:
   seqnames    ranges strand
      <Rle> <IRanges>  <Rle>
  -------
  seqinfo: no sequences

$NRXN1
GRanges object with 0 ranges and 0 metadata columns:
   seqnames    ranges strand
      <Rle> <IRanges>  <Rle>
  -------
  seqinfo: no sequences

...
<25068 more elements>

In [14]:
# I need to retrieve genome region info of genes:

library(AnnotationHub)

Loading required package: BiocFileCache

Loading required package: dbplyr


Attaching package: ‘AnnotationHub’


The following object is masked from ‘package:Biobase’:

    cache




In [15]:
ah <- AnnotationHub()

In [16]:
edb <- ah[["AH98047"]]  #"EnsDb.Hsapiens.v105"

downloading 1 resources

retrieving 1 resource

loading from cache

require(“ensembldb”)



In [17]:
gr <- genes(edb, columns = c("gene_id", "gene_name"))

In [18]:
common_genes <- na.omit(intersect(gr$gene_name, rownames(GeneExpressionMatrix)))

In [19]:
GeneExpressionMatrix <- GeneExpressionMatrix[common_genes,]

In [20]:
rowRanges(GeneExpressionMatrix) <- gr[match(common_genes, gr$gene_name)]

In [21]:
rowRanges(GeneExpressionMatrix)

GRanges object with 20667 ranges and 2 metadata columns:
                  seqnames            ranges strand |         gene_id
                     <Rle>         <IRanges>  <Rle> |     <character>
  ENSG00000279457        1     185217-195411      - | ENSG00000279457
  ENSG00000237491        1     778747-810065      + | ENSG00000237491
  ENSG00000177757        1     817371-819837      + | ENSG00000177757
  ENSG00000230368        1     868071-876903      - | ENSG00000230368
  ENSG00000223764        1     916865-921016      - | ENSG00000223764
              ...      ...               ...    ... .             ...
  ENSG00000198692        Y 20575776-20593154      + | ENSG00000198692
  ENSG00000280969        Y 20756108-20781032      + | ENSG00000280969
  ENSG00000169789        Y 22490397-22514637      + | ENSG00000169789
  ENSG00000188120        Y 23129355-23199010      - | ENSG00000188120
  ENSG00000205944        Y 23219447-23291356      + | ENSG00000205944
                    gene_name
   

In [22]:
GeneExpressionMatrix

class: SingleCellExperiment 
dim: 20667 49470 
metadata(0):
assays(3): counts logcounts scaledata
rownames(20667): ENSG00000279457 ENSG00000237491 ... ENSG00000188120
  ENSG00000205944
rowData names(2): gene_id gene_name
colnames(49470): 10X280_1:CAGATTCAGCAGCTCA 10X365_2:ACCAATATCAATGACC
  ... 10X346_4:CGCTGTGCACGTAATT 10X406_4:GAGAAACGTGGTGAGA
colData names(89): orig.ident nCount_RNA ... main_cell_types ident
reducedDimNames(0):
mainExpName: RNA
altExpNames(0):

In [23]:
# We then convert ATAC matrix to PeakMatrix. After conversion to SingleCellExperiment, 
# the peak positions appear as rownames and must be converted to GRanges.

In [24]:
mannens_et_al_seurat

An object of class Seurat 
430192 features across 49470 samples within 2 assays 
Active assay: RNA (25071 features, 5000 variable features)
 3 layers present: counts, data, scale.data
 1 other assay present: peaks

In [25]:
PeakMatrix <- as.SingleCellExperiment(mannens_et_al_seurat, assay="peaks")

In [26]:
PeakMatrix

class: SingleCellExperiment 
dim: 405121 49470 
metadata(0):
assays(2): counts logcounts
rownames(405121): chr10-100006329-100006730 chr10-100009751-100010152
  ... chrY-7725786-7726187 chrY-7729552-7729953
rowData names(0):
colnames(49470): 10X280_1:CAGATTCAGCAGCTCA 10X365_2:ACCAATATCAATGACC
  ... 10X346_4:CGCTGTGCACGTAATT 10X406_4:GAGAAACGTGGTGAGA
colData names(89): orig.ident nCount_RNA ... main_cell_types ident
reducedDimNames(0):
mainExpName: peaks
altExpNames(0):

In [27]:
PeakMatrix |> rowRanges()

GRangesList object of length 405121:
$`chr10-100006329-100006730`
GRanges object with 0 ranges and 0 metadata columns:
   seqnames    ranges strand
      <Rle> <IRanges>  <Rle>
  -------
  seqinfo: no sequences

$`chr10-100009751-100010152`
GRanges object with 0 ranges and 0 metadata columns:
   seqnames    ranges strand
      <Rle> <IRanges>  <Rle>
  -------
  seqinfo: no sequences

$`chr10-100016741-100017142`
GRanges object with 0 ranges and 0 metadata columns:
   seqnames    ranges strand
      <Rle> <IRanges>  <Rle>
  -------
  seqinfo: no sequences

...
<405118 more elements>

In [28]:
peak_position <- strsplit(rownames(PeakMatrix), split = "-")

In [29]:
peak_position |> head()

[[1]]
[1] "chr10"     "100006329" "100006730"

[[2]]
[1] "chr10"     "100009751" "100010152"

[[3]]
[1] "chr10"     "100016741" "100017142"

[[4]]
[1] "chr10"     "100019766" "100020167"

[[5]]
[1] "chr10"     "100020276" "100020677"

[[6]]
[1] "chr10"     "100020877" "100021278"

In [35]:
sapply(peak_position,"[",1) |> head()

[1] "chr10" "chr10" "chr10" "chr10" "chr10" "chr10"

In [36]:
sapply(peak_position,"[",2) |> head()

[1] "100006329" "100009751" "100016741" "100019766" "100020276" "100020877"

In [37]:
gr <- GRanges(
    seqnames = sapply(peak_position,"[",1),
    ranges = IRanges(start = as.numeric(sapply(peak_position,"[",2)), 
                     end = as.numeric(sapply(peak_position,"[",3)))
)

In [38]:
rowRanges(PeakMatrix) <- gr

In [39]:
PeakMatrix

class: SingleCellExperiment 
dim: 405121 49470 
metadata(0):
assays(2): counts logcounts
rownames: NULL
rowData names(0):
colnames(49470): 10X280_1:CAGATTCAGCAGCTCA 10X365_2:ACCAATATCAATGACC
  ... 10X346_4:CGCTGTGCACGTAATT 10X406_4:GAGAAACGTGGTGAGA
colData names(89): orig.ident nCount_RNA ... main_cell_types ident
reducedDimNames(0):
mainExpName: peaks
altExpNames(0):

In [40]:
PeakMatrix |> rowRanges()

GRanges object with 405121 ranges and 0 metadata columns:
           seqnames              ranges strand
              <Rle>           <IRanges>  <Rle>
       [1]    chr10 100006329-100006730      *
       [2]    chr10 100009751-100010152      *
       [3]    chr10 100016741-100017142      *
       [4]    chr10 100019766-100020167      *
       [5]    chr10 100020276-100020677      *
       ...      ...                 ...    ...
  [405117]     chrY     7703790-7704191      *
  [405118]     chrY     7714477-7714878      *
  [405119]     chrY     7724233-7724634      *
  [405120]     chrY     7725786-7726187      *
  [405121]     chrY     7729552-7729953      *
  -------
  seqinfo: 24 sequences from an unspecified genome; no seqlengths

In [41]:
here::here()

[1] "/fast/AG_Bunina/Yusuf/Project_Endothelial_and_Stroke/Datasets/Chromatin_and_Gene_Exp/2024_C_A_Mannens_C_et_al/04_02_25"

In [42]:
GeneExpressionMatrix |> saveRDS(here::here('r_objects', 'GeneExpressionMatrix.RDS'))

In [43]:
PeakMatrix |> saveRDS(here::here('r_objects', 'PeakMatrix.RDS'))

In [50]:
.libPaths() |> list.files() |> _[120:300] |> sort()

[1] "bayestestR"       "beachmat"         "beeswarm"        
  [4] "bit"              "bit64"            "bitops"          
  [7] "blob"             "bluster"          "boot"            
 [10] "broom"            "broom.helpers"    "bslib"           
 [13] "caTools"          "cachem"           "callr"           
 [16] "car"              "carData"          "cards"           
 [19] "caret"            "cellranger"       "checkmate"       
 [22] "chromVAR"         "circlize"         "class"           
 [25] "cli"              "clipr"            "clock"           
 [28] "clue"             "cluster"          "clusterProfiler" 
 [31] "coda"             "codetools"        "colorspace"      
 [34] "commonmark"       "compiler"         "conflicted"      
 [37] "coro"             "corrplot"         "cowplot"         
 [40] "cpp11"            "crayon"           "crosstalk"       
 [43] "curl"             "data.table"       "datasets"        
 [46] "datawizard"       "dbplyr"           "deldir"          
 [49] "diagram"          "dichromat"        "digest"          
 [52] "doBy"             "doParallel"       "dotCall64"       
 [55] "downloader"       "dplyr"            "dqrng"           
 [58] "dtplyr"           "e1071"            "edgeR"           
 [61] "ellmer"           "enrichplot"       "ensembldb"       
 [64] "entropy"          "epiregulon"       "epiregulon.extra"
 [67] "evaluate"         "fansi"            "farver"          
 [70] "fastDummies"      "fastmap"          "fastmatch"       
 [73] "fgsea"            "filelock"         "fitdistrplus"    
 [76] "fontawesome"      "forcats"          "foreach"         
 [79] "formatR"          "fs"               "futile.logger"   
 [82] "futile.options"   "future"           "future.apply"    
 [85] "gargle"           "gdsfmt"           "generics"        
 [88] "ggbeeswarm"       "ggforce"          "ggformula"       
 [91] "ggfun"            "ggnewscale"       "ggplot2"         
 [94] "ggplotify"        "ggpointdensity"   "ggprism"         
 [97] "ggpubr"           "ggraph"           "ggrastr"         
[100] "ggrepel"          "ggridges"         "ggsci"           
[103] "ggsignif"         "ggstats"          "ggtree"          
[106] "glmnet"           "glmnetUtils"      "globals"         
[109] "glue"             "goftest"          "googledrive"     
[112] "googlesheets4"    "gower"            "gplots"          
[115] "grDevices"        "graph"            "graphics"        
[118] "graphlayouts"     "grid"             "gridExtra"       
[121] "gridGraphics"     "grr"              "gson"            
[124] "gtable"           "gtools"           "hardhat"         
[127] "haven"            "hdf5r"            "here"            
[130] "highr"            "hms"              "htmltools"       
[133] "htmlwidgets"      "httpuv"           "httr"            
[136] "httr2"            "ica"              "ids"             
[139] "igraph"           "insight"          "ipred"           
[142] "irlba"            "isoband"          "iterators"       
[145] "janitor"          "jose"             "jquerylib"       
[148] "jsonlite"         "kernlab"          "knitr"           
[151] "labeling"         "labelled"         "lambda.r"        
[154] "later"            "lattice"          "lava"            
[157] "lazyeval"         "leiden"           "leidenbase"      
[160] "lifecycle"        "limma"            "listenv"         
[163] "lme4"             "lmtest"           "locfit"          
[166] "lubridate"        "magrittr"         "mapproj"         
[169] "maps"             "matrixStats"      "memoise"         
[172] "metapod"          "methods"          "mgcv"            
[175] "microbenchmark"   "mime"             "miniUI"          
[178] "minqa"            "mixtools"         "modelr"          
[181] "mosaic"

In [51]:
c("irlba", "uwot", "RcppHNSW", "igraph", "BiocManager", "remotes", "ggplot2") %in% list.files(.libPaths())

[1] TRUE TRUE TRUE TRUE TRUE TRUE TRUE

In [52]:
c("BSgenome.Hsapiens.UCSC.hg38") %in% list.files(.libPaths())

[1] TRUE

In [53]:
c("motifmatchr") %in% list.files(.libPaths())

[1] TRUE

In [54]:
c("chromVARmotifs") %in% list.files(.libPaths())

[1] FALSE

In [55]:
remotes::install_github(c("GreenleafLab/chromVARmotifs"), repos=BiocManager::repositories())


'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    BioCsoft: https://bioconductor.org/packages/3.19/bioc
    BioCann: https://bioconductor.org/packages/3.19/data/annotation
    BioCexp: https://bioconductor.org/packages/3.19/data/experiment
    BioCworkflows: https://bioconductor.org/packages/3.19/workflows
    BioCbooks: https://bioconductor.org/packages/3.19/books
    CRAN: https://cran.r-project.org



rjson        (0.2.22    -> 0.2.23   ) [CRAN]
XML          (3.99-0.17 -> 3.99-0.18) [CRAN]
GenomicRa... (1.56.1    -> 1.56.2   ) [CRAN]
R.oo         (1.26.0    -> 1.27.0   ) [CRAN]
poweRlaw     (0.80.0    -> 1.0.0    ) [CRAN]
RSQLite      (2.3.7     -> 2.3.9    ) [CRAN]


Installing 6 packages: rjson, XML, GenomicRanges, R.oo, poweRlaw, RSQLite

Installing packages into '/fast/AG_Bunina/Yusuf/Project_Endothelial_and_Stroke/Datasets/Chromatin_and_Gene_Exp/2024_C_A_Mannens_C_et_al/04_02_25/renv/library/linux-rhel-9.4/R-4.4/x86_64-unknown-linux-gnu'
(as 'lib' is unspecified)

Warning message in i.p(...):
"installation of package 'XML' had non-zero exit status"
Running `R CMD build`...



* checking for file ‘/tmp/RtmpBMJ5D3/remotes37818e2ad30784/GreenleafLab-chromVARmotifs-38bed55/DESCRIPTION’ ... OK
* preparing ‘chromVARmotifs’:
* checking DESCRIPTION meta-information ... OK
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
* building ‘chromVARmotifs_0.2.0.tar.gz’


Installing package into '/fast/AG_Bunina/Yusuf/Project_Endothelial_and_Stroke/Datasets/Chromatin_and_Gene_Exp/2024_C_A_Mannens_C_et_al/04_02_25/renv/library/linux-rhel-9.4/R-4.4/x86_64-unknown-linux-gnu'
(as 'lib' is unspecified)



In [61]:
mannens_et_al_seurat

An object of class Seurat 
430192 features across 49470 samples within 2 assays 
Active assay: RNA (25071 features, 5000 variable features)
 3 layers present: counts, data, scale.data
 1 other assay present: peaks

In [62]:
mannens_et_al_seurat@assays$peaks

ChromatinAssay data with 405121 features for 49470 cells
Variable features: 101285 
Genome: 
Annotation present: TRUE 
Motifs present: FALSE 
Fragment files: 0 

In [63]:
mannens_et_al_seurat@assays$peaks$data |> head()

  [[ suppressing 34 column names '10X280_1:CAGATTCAGCAGCTCA', '10X365_2:ACCAATATCAATGACC', '10X365_2:ATGGTGCGTCACCTAT' ... ]]



6 x 49470 sparse Matrix of class "dgCMatrix"
                                                                      
chr10-100006329-100006730 . . . . . . . . .         . . . . . .       
chr10-100009751-100010152 . . . . . . . . 0.8201684 . . . . . 1.070958
chr10-100016741-100017142 . . . . . . . . .         . . . . . .       
chr10-100019766-100020167 . . . . . . . . .         . . . . . .       
chr10-100020276-100020677 . . . . . . . . .         . . . . . .       
chr10-100020877-100021278 . . . . . . . . .         . . . . . .       
                                                                         
chr10-100006329-100006730 .         . . .         .        .        . . .
chr10-100009751-100010152 0.7369955 . . 0.7147449 1.023804 .        . . .
chr10-100016741-100017142 .         . . .         .        .        . . .
chr10-100019766-100020167 .         . . .         .        .        . . .
chr10-100020276-100020677 .         . . .         .        .        . . .
chr10-10002087

IMPPORTANT !! Signac runs SVD on the default assay if not explicitly stated !!!
Always set the the default assay to "peaks" or set the assay parameter explicitly !!!!

==============================================

RunSVD function crashed during runon the peaks assay.